Etape préliminaire 

Avant de commencer le projet, il faut installer les packages dont nous allons avoir besoin

In [1]:
import requests
import pandas as pd
import zipfile
import os

Etape 1 : Importation des données

Afin que cette étape se déroule bien, il faut récupérer la base DS_BPE_2024_data.csv préalablement importée sur S3.

In [ ]:
# Etape 1.1 : téléchargement des données électorales par API

url = "https://www.data.gouv.fr/api/1/datasets/r/ab337c6f-e7e8-4981-843c-45052b71096b"

response = requests.get(url)
if response.status_code == 200:  # si tout va bien on obtient 200 -> le code continue
    with open("results_legisl.csv", "wb") as file:
        file.write(response.content)
    print("Téléchargement results_legisl.csv réussi !")
else:
    print("Erreur HTTP :", response.status_code)

# path = os.path.expanduser("~/work/results_legisl.csv")
# print(os.path.exists(path), os.path.getsize(path))
# with open(path, "rb") as f:
#     print(f.read(200))
# Ce code a mis en évidence que results_legisl est en fait un zip

path = "/home/onyxia/work/results_legisl.csv"  # en réalité .zip !

with zipfile.ZipFile(path, "r") as z:
    files = z.namelist()
    print("Contenu du ZIP :", files)   # permet de voir ce que contient le zip


path = os.path.expanduser("~/work/results_legisl.csv")  # car c'est un .xlsx (ligne précédente)

# Renommer pour clarifier
os.rename(path, path.replace(".csv", ".xlsx"))
path = path.replace(".csv", ".xlsx")

# On lit le fichier excel avec pandas
df_legisl = pd.read_excel(path, sheet_name=0)  # sheet_name=0 pour la première feuille

print("Le dataframe df_legisl peut désormais être traité comme un data frame classique avec pandas")

"""#  Etape 2 : téléchargement des données infrastructures par API

# On a d'abord essayé en téléchargeant comme un csv mais cela ne fonctionnait pas

url = "https://api.insee.fr/melodi/data/DS_BPE"

response = requests.get(url)
#if response.status_code == 200:
#    data_json = response.json()
    print("Téléchargement JSON infrastructures réussi")

    # Vérifier la clé 'observations'
    observations = data_json['observations']
    print(f"Nombre d'observations : {len(observations)}")

    # Convertir en DataFrame
    df_infra = pd.DataFrame(observations)

    # Afficher les premières lignes
    print(df_infra.head())

    # Sauvegarder si besoin
    df_infra.to_csv("infrastruct.csv", index=False, sep=';')

else:
    print("Erreur HTTP :", response.status_code)"""
# Etape 2 : récupération des données sur les infrastructures

df_infrastruct = pd.read_csv("DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;
print("Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas")

Etape 2 : Préparation du dataframe que nous allons utiliser

In [ ]:
# Préparation et nettoyage des bases de données

print("Informations sur le dataframe df_legisl")
print(df_legisl.columns)
print(df_legisl.size)
print(df_legisl.head())

print("Informations sur le dataframe df_infrastruct")
print(df_infrastruct.columns)
print(df_infrastruct.size)
print(df_infrastruct.head())